# 1. Import the necessary libraries for you to be able to query 4square API

In [1]:
import requests
import json
import pandas as pd
from getpass import getpass

# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [ ]:
import requests
import getpass

api_key = getpass.getpass("Enter your API key: ")

url = "https://api.foursquare.com/v3/places/search"
headers = {
    "Accept": "application/json",
    "Authorization": api_key
}

# Make the request to the Foursquare API
response = requests.get(url, headers=headers)


In [15]:
import requests

params = {
    "query": "coffee",  
    "near": "Lisbon",   
    "limit": 10         
}

response = requests.get(url, headers=headers, params=params)

data = response.json()

if "results" in data:
    for place in data["results"]:
        name = place.get("name", "Name not available")
        location = place.get("location", {})
        address = location.get("formatted_address", "Address not available")
        print(f"Name: {name}\nAddress: {address}\n")
else:
    print("No results found.")



Name: Café da Garagem
Address: Rua Costa Castelo 75, 1100-178 Lisboa

Name: Damas
Address: Rua da Voz do Operário 60, 1070-087 Lisboa

Name: Copenhagen Coffee Lab
Address: Escolas Gerais, 34, 1100-213 Lisboa

Name: Pastelaria Santo António
Address: Rua St António 88, 4800-162 Lisboa

Name: Fabrica da Nata
Address: Rua Augusta, 1100-052 Lisboa

Name: Quiosque Portas do Sol
Address: Lisboa

Name: Copenhagen Coffee Lab
Address: Rua dos Douradores, Lisbon Portugal (Rua de Santa Justa), 1100-230 Lisboa

Name: Curva
Address: Rua Damasceno Monteiro 108D, Lisboa

Name: Fábrica da Nata
Address: Praça dos Restauradores nº 62, 1250-110 Lisboa

Name: Fábrica Coffee Roasters
Address: Rua das Portas de Santo Antão 136, 1150-265 Lisboa



# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [ ]:

params = {
    "query": "coffee",  
    "near": "Lisbon",   
    "limit": 5          
}

response = requests.get(url, headers=headers, params=params)

data = response.json()

if "results" in data:
    print("5 Closest Coffee Places:")
    for place in data["results"]:
        name = place.get("name", "Name not available")
        location = place.get("location", {})
        address = location.get("formatted_address", "Address not available")
        print(f"Name: {name}, Address: {address}")
else:
    print("No coffee places found.")


5 Closest Coffee Places:
Name: Café da Garagem, Address: Rua Costa Castelo 75, 1100-178 Lisboa
Name: Damas, Address: Rua da Voz do Operário 60, 1070-087 Lisboa
Name: Copenhagen Coffee Lab, Address: Escolas Gerais, 34, 1100-213 Lisboa
Name: Pastelaria Santo António, Address: Rua St António 88, 4800-162 Lisboa
Name: Fabrica da Nata, Address: Rua Augusta, 1100-052 Lisboa


# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [ ]:


def foursquare_places(venue, coordinates):
    pass
    
    params = {
        "query": venue,       
        "ll": coordinates,    
        "limit": 10           
    }
    
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    
    results = []
    if "results" in data:
        for place in data["results"]:
            name = place.get("name", "Name not available")
            geocode = place.get("geocodes", {}).get("main", {})
            lat = geocode.get("latitude", "Latitude not available")
            lng = geocode.get("longitude", "Longitude not available")
            results.append({"name": name, "latitude": lat, "longitude": lng})
    
    return results

venues = foursquare_places("Starbucks", "38.7169,-9.1399")  
for venue in venues:
    print(f"Name: {venue['name']}, Latitude: {venue['latitude']}, Longitude: {venue['longitude']}")


Name: Starbucks, Latitude: 38.71449, Longitude: -9.140825
Name: Starbucks, Latitude: 38.710633, Longitude: -9.13938
Name: Starbucks, Latitude: 38.712352, Longitude: -9.13872
Name: Starbucks, Latitude: 38.770219, Longitude: -9.128875
Name: Starbucks, Latitude: 38.728264, Longitude: -9.218769
Name: Starbucks ECI Lisboa, Latitude: 38.733857, Longitude: -9.153991
Name: Starbucks, Latitude: 38.771343, Longitude: -9.130866
Name: Starbucks, Latitude: 38.768788, Longitude: -9.095998
Name: Starbucks, Latitude: 38.714539, Longitude: -9.140885
Name: Starbucks, Latitude: 38.71449, Longitude: -9.140825


# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [ ]:
import os
import json

coffee_shops = [
    {"name": "Starbucks", "latitude": 38.71449, "longitude": -9.140825},
    {"name": "Starbucks", "latitude": 38.710633, "longitude": -9.13938},
    {"name": "Starbucks", "latitude": 38.712352, "longitude": -9.13872},
    {"name": "Starbucks", "latitude": 38.770219, "longitude": -9.128875},
    {"name": "Starbucks", "latitude": 38.728264, "longitude": -9.218769},
    {"name": "Starbucks ECI Lisboa", "latitude": 38.733857, "longitude": -9.153991},
    {"name": "Starbucks", "latitude": 38.771343, "longitude": -9.130866},
    {"name": "Starbucks", "latitude": 38.768788, "longitude": -9.095998},
    {"name": "Starbucks", "latitude": 38.714539, "longitude": -9.140885},
    {"name": "Starbucks", "latitude": 38.71449, "longitude": -9.140825},
    {"name": "Café da Garagem", "address": "Rua Costa Castelo 75, 1100-178 Lisboa"},
    {"name": "Damas", "address": "Rua da Voz do Operário 60, 1070-087 Lisboa"},
    {"name": "Copenhagen Coffee Lab", "address": "Escolas Gerais, 34, 1100-213 Lisboa"},
    {"name": "Pastelaria Santo António", "address": "Rua St António 88, 4800-162 Lisboa"},
    {"name": "Fabrica da Nata", "address": "Rua Augusta, 1100-052 Lisboa"},
    {"name": "Quiosque Portas do Sol", "address": "Lisboa"},
    {"name": "Copenhagen Coffee Lab", "address": "Rua dos Douradores, Lisbon Portugal (Rua de Santa Justa), 1100-230 Lisboa"},
    {"name": "Curva", "address": "Rua Damasceno Monteiro 108D, Lisboa"},
    {"name": "Fábrica da Nata", "address": "Unknown Address"},
    {"name": "Fábrica Coffee Roasters", "address": "Rua das Portas de Santo Antão 136, 1150-265 Lisboa"}
]

output_folder = "data"
output_file = os.path.join(output_folder, "coffee_shops.json")

os.makedirs(output_folder, exist_ok=True)

with open(output_file, "w") as f:
    json.dump(coffee_shops, f, indent=4)

print(f"Data successfully exported to {output_file}")



Data successfully exported to data\coffee_shops.json


# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [ ]:
# your code here